In [74]:
from lb.helpers import *
import requests
import time

BASE_URL = "http://localhost:8000"

### remove lb,servers running

In [75]:
# remove_servers(["lb"])
# server_list = get_servers("serverimg")
# remove_servers(server_list)


In [76]:
response = requests.post(f"{BASE_URL}/init",json={
  "N": 3,
  "schema": {
    "columns": [
      "Stud_id",
      "Stud_name",
      "Stud_marks"
    ],
    "dtypes": [
      "Number",
      "String",
      "Number"
    ]
  },
  "shards": [
    {
      "Stud_id_low": 0,
      "Shard_id": "sh1",
      "Shard_size": 4096
    },
    {
      "Stud_id_low": 4096,
      "Shard_id": "sh2",
      "Shard_size": 4096
    },
    {
      "Stud_id_low": 8192,
      "Shard_id": "sh3",
      "Shard_size": 4096
    }
  ],
  "servers": {
    "Server0": [
      "sh1",
      "sh2"
    ],
    "Server1": [
      "sh2",
      "sh3"
    ],
    "Server2": [
      "sh1",
      "sh3"
    ]
  }
})

print(response.json())

{'message': 'Configured Database', 'status': 'success'}


In [77]:
response = requests.get(f"{BASE_URL}/status")

print(response.json())

{'N': 3, 'schema': {'columns': ['Stud_id', 'Stud_name', 'Stud_marks'], 'dtypes': ['Number', 'String', 'String']}, 'shards': [{'Stud_id_low': 0, 'Shard_id': 'sh1', 'Shard_size': 4096}, {'Stud_id_low': 4096, 'Shard_id': 'sh2', 'Shard_size': 4096}, {'Stud_id_low': 8192, 'Shard_id': 'sh3', 'Shard_size': 4096}], 'servers': {'Server0': ['sh1', 'sh2'], 'Server1': ['sh2', 'sh3'], 'Server2': ['sh1', 'sh3']}}


In [78]:
response = requests.post(f"{BASE_URL}/write",json={
    "data": [{"Stud_id":2255,"Stud_name":"GHI","Stud_marks":27},
            {"Stud_id":5000,"Stud_name":"JKBFSFS","Stud_marks":56},
            {"Stud_id":9000,"Stud_name":"sjk","Stud_marks":46}
            ]
})
print(response.json())

{'message': '3 Data entries added', 'status': 'success'}


In [79]:
response = requests.post(f"{BASE_URL}/read", json={
    "Stud_id": {"low":1000, "high":10000}
})
print(response.json())

{'shards_queried': ['sh1', 'sh2', 'sh3'], 'data': [[2255, 'GHI', '27'], [5000, 'JKBFSFS', '56'], [9000, 'sjk', '46']], 'status': 'success'}


In [80]:
response = requests.put(f"{BASE_URL}/update",json={
    "Stud_id":2255, # this should be an integer
    "data": {"Stud_id":2255,"Stud_name":"GHI","Stud_marks":30}
})

print(response.json())

response = requests.post(f"{BASE_URL}/read", json={
    "Stud_id": {"low":1000, "high":10000}
})
print(response.json())

{'message': 'Data entry for Stud_id:2255 updated', 'status': 'success'}
{'shards_queried': ['sh1', 'sh2', 'sh3'], 'data': [[2255, 'GHI', '30'], [5000, 'JKBFSFS', '56'], [9000, 'sjk', '46']], 'status': 'success'}


In [81]:
response = requests.delete(f"{BASE_URL}/del",json={
    "Stud_id":2255
})
print(response.json())

response = requests.post(f"{BASE_URL}/read", json={
    "Stud_id": {"low":1000, "high":4000}
})
print(response.json())

{'message': 'Data entry for Stud_id:2255 deleted', 'status': 'success'}
{'shards_queried': ['sh1'], 'data': [], 'status': 'success'}


In [82]:
response = requests.delete(f"{BASE_URL}/rm",json={
    "n" :1,
     "servers" : ["Server1"]
})
print(response.json())

response = requests.get(f"{BASE_URL}/status")
print(response.json())


{'message': {'N': 2, 'servers': ['Server1']}, 'status': 'successful'}
{'N': 2, 'schema': {'columns': ['Stud_id', 'Stud_name', 'Stud_marks'], 'dtypes': ['Number', 'String', 'String']}, 'shards': [{'Stud_id_low': 0, 'Shard_id': 'sh1', 'Shard_size': 4096}, {'Stud_id_low': 4096, 'Shard_id': 'sh2', 'Shard_size': 4096}, {'Stud_id_low': 8192, 'Shard_id': 'sh3', 'Shard_size': 4096}], 'servers': {'Server0': ['sh1', 'sh2'], 'Server2': ['sh1', 'sh3']}}


In [83]:
response = requests.post(f"{BASE_URL}/add",json={
"n" : 2,
"new_shards":[{"Stud_id_low":12288, "Shard_id": "sh5", "Shard_size":4096}],

"servers" : {"Server4":["sh3","sh5"], 
"Server5":["sh2","sh5"]}
})
print(response.json())

response = requests.get(f"{BASE_URL}/status")
print(response.json())


{'N': 4, 'message': 'Added Server4, Server5,', 'status': 'success'}
{'N': 4, 'schema': {'columns': ['Stud_id', 'Stud_name', 'Stud_marks'], 'dtypes': ['Number', 'String', 'String']}, 'shards': [{'Stud_id_low': 0, 'Shard_id': 'sh1', 'Shard_size': 4096}, {'Stud_id_low': 4096, 'Shard_id': 'sh2', 'Shard_size': 4096}, {'Stud_id_low': 8192, 'Shard_id': 'sh3', 'Shard_size': 4096}, {'Stud_id_low': 12288, 'Shard_id': 'sh5', 'Shard_size': 4096}], 'servers': {'Server0': ['sh1', 'sh2'], 'Server2': ['sh1', 'sh3'], 'Server4': ['sh3', 'sh5'], 'Server5': ['sh2', 'sh5']}}


### Crashing server and checking the server respawn

In [85]:

response = requests.get(f"{BASE_URL}/status")
print(response.json())

{'N': 4, 'schema': {'columns': ['Stud_id', 'Stud_name', 'Stud_marks'], 'dtypes': ['Number', 'String', 'String']}, 'shards': [{'Stud_id_low': 0, 'Shard_id': 'sh1', 'Shard_size': 4096}, {'Stud_id_low': 4096, 'Shard_id': 'sh2', 'Shard_size': 4096}, {'Stud_id_low': 8192, 'Shard_id': 'sh3', 'Shard_size': 4096}, {'Stud_id_low': 12288, 'Shard_id': 'sh5', 'Shard_size': 4096}], 'servers': {'Server2': ['sh1', 'sh3'], 'Server4': ['sh3', 'sh5'], 'Server5': ['sh2', 'sh5'], 'new_Server0': ['sh1', 'sh2']}}
